#### Init

In [2]:
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from glob import glob
import os
import cv2


from landmarks.landmarks_extraction import load_images_bgr, Video_preprocessing

### Video basic

In [55]:
img_size = (658, 492)
crop_size = (300, 300)


crop_margin = np.array(crop_size)//2


def create_img_crop(img,lm):
    max_p = np.max(lm,axis=0)
    min_p = np.min(lm,axis=0)
    center = min_p + (max_p-min_p)//2
    p_up = center-crop_margin
    p_down = center+crop_margin


    # set margin
    if p_up[0] < 0:
        p_down[0] -= p_up[0]
        p_up[0] = 0
    if p_up[1] < 0:
        p_down[1] -= p_up[1]
        p_up[1] = 0 
    if p_down[0] > img_size[0]:
        p_up[0] -= p_down[0] - img_size[0]
        p_down[0] = img_size[0]
    if p_down[1] > img_size[1]:
        p_up[1] -= p_down[1] - img_size[1]
        p_down[1] = img_size[1]


    return img[p_up[1]:p_down[1],p_up[0]:p_down[0],:]

In [56]:
folder_castphys = Path("D:/TFG_ALEX/castphys_60")
save_folder = Path("D:/TFG_ALEX/castphys_crop")

options = {}
preprocessing = Video_preprocessing(options)
fps = 60

ids = [int(patient.split('_')[1]) for patient in glob("Patient_*", root_dir=folder_castphys)]
id = ids[0]
Patient_path = folder_castphys/ f"Patient_{id}"
list_videos = glob("Q*",root_dir=Patient_path)# + glob("E*",root_dir=Patient_path)
v_name = list_videos[0]
img_path = Patient_path / v_name / "imgs"


video = load_images_bgr(img_path, [])
pred_landmarks = preprocessing.landmarks_extraction(video)


video_crop = [create_img_crop(frm,lm) for frm, lm in zip(video,pred_landmarks)]


video_save = cv2.VideoWriter(str(save_folder / f"video_{id}_{v_name}_full.mp4"),
                             cv2.VideoWriter_fourcc(*'mp4v'),fps,crop_size)
for i in range(0,len(video_crop)): #6
    video_save.write(video_crop[i])
video_save.release()

### Video Smoothing

In [57]:
img_size = (658, 492)
crop_size = (300, 300)
sr = (30,30) # Smoothing range

crop_margin = np.array(crop_size)//2


def create_img_crop_smoot(img,center_list):
    center = np.mean(center_list,axis=0,dtype=int)
    p_up = center-crop_margin
    p_down = center+crop_margin


    # set margin
    if p_up[0] < 0:
        p_down[0] -= p_up[0]
        p_up[0] = 0
    if p_up[1] < 0:
        p_down[1] -= p_up[1]
        p_up[1] = 0 
    if p_down[0] > img_size[0]:
        p_up[0] -= p_down[0] - img_size[0]
        p_down[0] = img_size[0]
    if p_down[1] > img_size[1]:
        p_up[1] -= p_down[1] - img_size[1]
        p_down[1] = img_size[1]


    return img[p_up[1]:p_down[1],p_up[0]:p_down[0],:]

In [58]:
folder_castphys = Path("D:/TFG_ALEX/castphys_60")
save_folder = Path("D:/TFG_ALEX/castphys_crop")

options = {}
preprocessing = Video_preprocessing(options)

fps = 60


ids = [int(patient.split('_')[1]) for patient in glob("Patient_*", root_dir=folder_castphys)]
id = ids[0]
Patient_path = folder_castphys/ f"Patient_{id}"
list_videos = glob("Q*",root_dir=Patient_path)# + glob("E*",root_dir=Patient_path)
v_name = list_videos[0]
img_path = Patient_path / v_name / "imgs"

video = load_images_bgr(img_path, [])
pred_landmarks = preprocessing.landmarks_extraction(video)

center_list = [np.min(lm,axis=0) + (np.max(lm,axis=0)-np.min(lm,axis=0))//2 for lm in pred_landmarks]

video_crop = [create_img_crop_smoot(frm,center_list[id-sr[0] if id-sr[0]>=0 else 0:id+sr[1]]) 
              for id, frm in enumerate(video)]

video_save = cv2.VideoWriter(str(save_folder / f"video_{id}_{v_name}_s.mp4"),
                             cv2.VideoWriter_fourcc(*'mp4v'),fps,crop_size)
for i in range(0,len(video_crop)): #6
    video_save.write(video_crop[i])
video_save.release()

### Video Static

In [38]:
folder_crop = Path("D:/TFG_ALEX/crop")
img_size = (658, 492)

def transform_column(df, col):
    return [[int(''.join(i[:-1])[1:]),int(i[-1][:-1])] for i in [e.split(' ') for e in df[col]]]

ids = [int(file.split('.')[0].split('_')[-1]) for file in glob("crop_lm_*", root_dir=folder_crop)]
ids.sort()

#id = ids[0]
max_all_size = []
for id in ids:
    df = pd.read_csv(folder_crop / f"crop_lm_{id}.csv")

    df['r_size'] = transform_column(df, 'r_size')
    df['p_up'] = transform_column(df, 'p_up')
    df['p_down'] = transform_column(df, 'p_down')
    #df['d_down'] = transform_column(df, 'd_down') ## Esta mal
    df.columns = ["video", "r_size", "p_up", "p_down", "d_down"]

    #max_r_size = [[(x_down)-x_up,(y_down)-y_up] 
    #               for [x_up,y_up],[x_down,y_down] in zip(df['p_up'],df['p_down'])]
    max_r_size = [[(x_down if x_down<img_size[0] else img_size[0])-x_up,
                   (y_down if y_down<img_size[1] else img_size[1])-y_up] 
                   for [x_up,y_up],[x_down,y_down] in zip(df['p_up'],df['p_down'])]
    max_all_size.append(np.max(max_r_size, axis=0))

#max_all_size
np.max(max_all_size, axis=0)


array([428, 335])

In [31]:
img_size = (658, 492)
crop_size = (450, 350)

crop_margin = np.array(crop_size)//2


def search_margin_static(center):
    p_up = center-crop_margin
    p_down = center+crop_margin

    # set margin
    if p_up[0] < 0:
        p_down[0] -= p_up[0]
        p_up[0] = 0
    if p_up[1] < 0:
        p_down[1] -= p_up[1]
        p_up[1] = 0 
    if p_down[0] > img_size[0]:
        p_up[0] -= p_down[0] - img_size[0]
        p_down[0] = img_size[0]
    if p_down[1] > img_size[1]:
        p_up[1] -= p_down[1] - img_size[1]
        p_down[1] = img_size[1]



    return p_up,p_down

In [35]:
folder_castphys = Path("D:/TFG_ALEX/castphys_60")
save_folder = Path("D:/TFG_ALEX/castphys_crop")

options = {}
preprocessing = Video_preprocessing(options)

fps = 60

ids = [int(patient.split('_')[1]) for patient in glob("Patient_*", root_dir=folder_castphys)]
id = ids[0]
Patient_path = folder_castphys/ f"Patient_{id}"
list_videos = glob("Q*",root_dir=Patient_path)# + glob("E*",root_dir=Patient_path)
v_name = list_videos[0]
img_path = Patient_path / v_name / "imgs"

video = load_images_bgr(img_path, [])
pred_landmarks = preprocessing.landmarks_extraction(video)

center = np.min(np.min(pred_landmarks,axis=0),axis=0) + (np.max(np.max(pred_landmarks,axis=0),axis=0)-np.min(np.min(pred_landmarks,axis=0),axis=0))//2
p_up,p_down = search_margin_static(center)

video_crop = [frm[p_up[1]:p_down[1],p_up[0]:p_down[0],:] for frm in video]

video_save = cv2.VideoWriter(str(save_folder / f"video_{id}_{v_name}_static.avi"),
                             cv2.VideoWriter_fourcc(*'FFV1'),fps,crop_size)
for i in range(0,len(video_crop)): #6
    video_save.write(video_crop[i])
video_save.release()